# Load environment variables

In [1]:
# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()

# Azure Credentials
subscription_key = os.getenv("AZURE_SUBSCRIPTION_KEY")
endpoint = os.getenv("AZURE_VISION_ENDPOINT")

# OpenAI Credentials
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_KEY")
deployment_name = 'gpt-4o'
api_version = '2023-12-01-preview'

# Import necessary libraries from the project

In [7]:
# Import necessary libraries from the project
from src.image_translator.azure_vision_translator import extract_line_bounding_boxes, extract_text_from_image
from src.text_translator.openai_translator import translate_text
from src.utils.image_utils import plot_annotated_image, save_bounding_boxes, load_bounding_boxes
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import cv2
import json

# Path to font file
FONT_PATH = "./fonts/NotoSans-Medium.ttf"

# Testing with Computer Vision Client

In [9]:
def retrieve_bounding_boxes_by_image_path(image_path):
    """
    Retrieve bounding boxes for a given image path by loading from a saved JSON file.
    
    Args:
        image_path (str): Path to the image file.
    
    Returns:
        list: List of bounding boxes and text data.
    """
    image_name = os.path.basename(image_path).split(".")[0]
    json_path = f"./bounding_boxes/{image_name}.json"
    if os.path.exists(json_path):
        return load_bounding_boxes(json_path)
    else:
        raise FileNotFoundError(f"Bounding box data not found for image: {image_path}")

# Function to process the image and annotate with bounding boxes and translated text
def process_and_translate_image(image_path, target_language):
    # Extract text and bounding boxes
    line_bounding_boxes = extract_line_bounding_boxes(image_path)

    # Display and save bounding boxes
    if line_bounding_boxes:
        for line_info in line_bounding_boxes:
            print(line_info)
        save_bounding_boxes(image_path, line_bounding_boxes)
    else:
        raise Exception("No text was recognized in the image.")

    # Generate translation prompt and get translated text
    text_data = [line['text'] for line in line_bounding_boxes]
    translated_text_data = translate_text(text_data, target_language)

    # Annotate image with translated text
    plot_annotated_image(image_path, line_bounding_boxes, translated_text_data)

# High-level function to translate image

In [11]:
# High-level function to translate image
def translate_image(image_path, language):
    line_bounding_boxes = retrieve_bounding_boxes_by_image_path(image_path)
    text_data = [line['text'] for line in line_bounding_boxes]
    translated_text_data = translate_text(text_data, language)
    plot_annotated_image(image_path, line_bounding_boxes, translated_text_data)

# Example usage
image_path = "../data/images/bicycle.png"
process_and_translate_image(image_path, "Spanish")

translate_image("../data/images/korean.png", "English")
translate_image("../data/images/microsoft1.png", "Malay")

ComputerVisionOcrErrorException: Operation returned an invalid status code 'PermissionDenied'